# Example: Ingest Real-Time Stock Data to iguazio NoSQL and TSDB
the following example function ingest real-time stock information from an internet service (World Trading Data) into iguazio platform.<br>
everytime the data is updated it updates a NoSQL table with the recent metadata and updates the TSDB with the new metrics (price and volume)

The same code can run inside a nuclio (serverless) function and be automatically triggered on a predefined schedule (cron) or through HTTP requests

## Initialization (not requiered in nuclio)
need to fill the following environment variables with real credentials.<br>
```
    API_TOKEN = <WorldTradingData API Token>
    V3IO_PASSWORD = <V3IO-Password>
    V3IO_USER = <V3IO-Username>
    V3IO_ADDRESS = <address of V3IO API end point>
    FRAMESD_URL = <V3IO Framesd URL (Columnar & TSDB API)>
```
obtain your free API token from [World Trading Data](https://www.worldtradingdata.com) <br>
> note: in nuclio environment variables and build dependencies are specified in the configuration tab, for notebooks they can be initialized from a file as implemented below.

### initialize nuclio emulation (this section will be ignored by nuclio nbconvert)

In [1]:
# nuclio: ignore
from nuclio import Context, Event
from v3io import env_fromfile
env_fromfile('stock_env.txt')
context = Context()

## Nuclio function implementation
this function can run in Jupyter or in nuclio (real-time serverless)

In [2]:
import json
import v3io
import requests
import os
import pandas as pd
import datetime
import v3io_frames as v3f

stocks = os.getenv('STOCK_LIST','GOOG,MSFT,AMZN,IBM,ORCL')
token = os.getenv('API_TOKEN') 
url = 'https://www.worldtradingdata.com/api/v1/stock?symbol={0}&api_token={1}'.format(stocks,token)
v3 = v3io.v3io(os.getenv('V3IO_ADDRESS'),os.getenv('V3IO_USER'),os.getenv('V3IO_PASSWORD'), 'bigdata')
client = v3f.Client()

# v3io update expression template 
expr_template = "symbol='{symbol}';name='{name}';currency='{currency}';exchange='{stock_exchange_short}';" + \
    "timezone='{timezone}';price={price};volume={volume};last_trade='{last_trade_time}'"

last_trade_times = {}

# nuclio habdler fuction 
def handler(context, event):
    
    # reading latest stock information 
    r = requests.get(url)
    stocks = []
    dates = []
    volumes = []
    prices = []
    
    for stock in r.json()['data']:
        
        symbol = stock['symbol']
        last = last_trade_times.get(symbol)
        date = datetime.datetime.strptime(stock['last_trade_time'], '%Y-%m-%d %H:%M:%S')
        
        # update the stocks table and TSDB metrics in case of new data 
        if not last or date > last:
            # update NoSQL table with stock data
            expr = expr_template.format(**stock)
            context.logger.debug_with('update expression', symbol=symbol, expr=expr)
            v3.updateitem('stocks',symbol, expr, '')
        
            # update time-series DB with price and volume metrics (use pandas dataframe with a single row, indexed by date)
            last_trade_times[symbol] = date 
            stocks += [symbol]
            dates +=[date]
            volumes += [float(stock['volume'])]
            prices += [float(stock['price'])]
               
    # write price and volume metrics to the Time-Series DB
    if len(stocks)>0:
        df = pd.DataFrame({'volume':volumes,'price': prices}, index=[dates,stocks], columns=['volume','price'])
        df.index.names=['time','symbol']
        context.logger.debug_with('writing data to TSDB', stocks=stocks)
        client.write(backend='tsdb', table='stock_metrics',dfs=df)
        
    return r.json()

## Function invocation
the following section simulates nuclio function invocation

In [3]:
event = Event(body='')
resp = handler(context, event)

2018-10-26 00:15:44,024 nuclio (D) update expression {"symbol": "AMZN", "expr": "symbol='AMZN';name='Amazon.com, Inc.';currency='USD';exchange='NASDAQ';timezone='EDT';price=1782.17;volume=9508483;last_trade='2018-10-25 16:00:02'"}
2018-10-26 00:15:44,208 nuclio (D) update expression {"symbol": "GOOG", "expr": "symbol='GOOG';name='Alphabet Inc Class C';currency='USD';exchange='NASDAQ';timezone='EDT';price=1095.57;volume=2452049;last_trade='2018-10-25 16:00:03'"}
2018-10-26 00:15:44,390 nuclio (D) update expression {"symbol": "IBM", "expr": "symbol='IBM';name='IBM Common Stock';currency='USD';exchange='NYSE';timezone='EDT';price=126.45;volume=10241580;last_trade='2018-10-25 16:02:15'"}
2018-10-26 00:15:44,559 nuclio (D) update expression {"symbol": "MSFT", "expr": "symbol='MSFT';name='Microsoft Corporation';currency='USD';exchange='NASDAQ';timezone='EDT';price=108.30;volume=61093099;last_trade='2018-10-25 16:00:02'"}
2018-10-26 00:15:44,739 nuclio (D) update expression {"symbol": "ORCL",